In [ ]:
#This program will train a model based on the 6_23_ground truth specular dataset creator

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision # Import torchvision to check version
import torchvision.models as models
from torchvision import transforms
import torch.optim as optim
import os
import cv2
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# --- Configuration (Keep as is) ---
DATASET_CONFIG = {
    "ORIGINAL_DIR": "dataset/original_frames",
    "GROUND_TRUTH_DIR": "dataset/ground_truth_frames",
    "TARGET_IMG_SIZE": (512, 512),
    "BATCH_SIZE": 4,
    "NUM_EPOCHS": 100,
    "LEARNING_RATE": 1e-4,
    "MODEL_SAVE_PATH": "specular_removal_unet_perceptual_emphasis.pth",
    "VGG_FEATURE_LAYERS": [3, 8, 17, 26],
    "PERCEPTUAL_LOSS_WEIGHT": 0.05
}
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")


# --- U-Net Components (Unchanged - as in previous correct version) ---
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels: mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(mid_channels), nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels), nn.ReLU(inplace=True)
        )
    def forward(self, x): return self.double_conv(x)

class Down(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(nn.MaxPool2d(2), DoubleConv(in_channels, out_channels))
    def forward(self, x): return self.maxpool_conv(x)

class Up(nn.Module):
    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)
    def forward(self, x1, x2):
        x1 = self.up(x1)
        diffY, diffX = x2.size()[2] - x1.size()[2], x2.size()[3] - x1.size()[3]
        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2, diffY // 2, diffY - diffY // 2])
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)

class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)
    def forward(self, x): return self.conv(x)

class UNet(nn.Module):
    def __init__(self, n_channels_in, n_channels_out, bilinear=True, base_features=64):
        super(UNet, self).__init__()
        self.inc = DoubleConv(n_channels_in, base_features)
        self.down1 = Down(base_features, base_features * 2)
        self.down2 = Down(base_features * 2, base_features * 4)
        self.down3 = Down(base_features * 4, base_features * 8)
        factor = 2 if bilinear else 1
        self.down4 = Down(base_features * 8, base_features * 16 // factor)
        self.up1 = Up(base_features * 16, base_features * 8 // factor, bilinear)
        self.up2 = Up(base_features * 8, base_features * 4 // factor, bilinear)
        self.up3 = Up(base_features * 4, base_features * 2 // factor, bilinear)
        self.up4 = Up(base_features * 2, base_features, bilinear)
        self.outc = OutConv(base_features, n_channels_out)
    def forward(self, x):
        x1 = self.inc(x); x2 = self.down1(x1); x3 = self.down2(x2); x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4); x = self.up2(x, x3); x = self.up3(x, x2); x = self.up4(x, x1)
        return torch.sigmoid(self.outc(x))

# --- Perceptual Loss (Corrected VGG loading) ---
class VGGPerceptualLoss(nn.Module):
    def __init__(self, feature_layers=None, use_input_norm=True, requires_grad=False):
        super(VGGPerceptualLoss, self).__init__()

        try:
            self.vgg_features = models.vgg19(weights=models.VGG19_Weights.IMAGENET1K_V1).features.to(DEVICE).eval()
            # print("Using torchvision VGG19 with 'weights' parameter.")
        except TypeError:
            # Fallback to `pretrained=True` for older torchvision versions
            self.vgg_features = models.vgg19(pretrained=True).features.to(DEVICE).eval()
            # print("Using torchvision VGG19 with 'pretrained=True' parameter (older torchvision).")

        if use_input_norm:
            self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                  std=[0.229, 0.224, 0.225])
        else:
            self.normalize = nn.Identity()

        if feature_layers is None:
            self.feature_layers_idx = [3, 8, 17, 26, 35] 
        else:
            self.feature_layers_idx = feature_layers

        if not requires_grad:
            for param in self.vgg_features.parameters():
                param.requires_grad = False
        self.loss_fn = nn.L1Loss()

    def get_features(self, x):
        x = self.normalize(x)
        features = []
        current_max_idx = max(self.feature_layers_idx) # Get max index once
        for i, layer in enumerate(self.vgg_features):
            x = layer(x)
            if i in self.feature_layers_idx:
                features.append(x)
            if i >= current_max_idx: # Optimization: stop if past last needed layer
                break
        return features

    def forward(self, pred, target):
        pred_features = self.get_features(pred)
        target_features = self.get_features(target)
        loss = 0
        for pf, tf in zip(pred_features, target_features):
            loss += self.loss_fn(pf, tf)
        return loss

# --- Dataset and DataLoader (Unchanged) ---
class SpecularRemovalDataset(Dataset):
    def __init__(self, original_dir, gt_dir, img_size):
        self.original_dir = original_dir
        self.gt_dir = gt_dir
        self.img_size = img_size
        self.original_files = sorted([f for f in os.listdir(original_dir) if f.endswith('.png')])
        self.transform = transforms.Compose([
            transforms.Resize(img_size),
            transforms.ToTensor(),
        ])
    def __len__(self): return len(self.original_files)
    def __getitem__(self, idx):
        fname = self.original_files[idx]
        original_img = Image.open(os.path.join(self.original_dir, fname)).convert("RGB")
        gt_img = Image.open(os.path.join(self.gt_dir, fname)).convert("RGB")
        return self.transform(original_img), self.transform(gt_img)

# --- Training Loop (Unchanged - as in previous correct version) ---
def train_model():
    dataset = SpecularRemovalDataset(
        DATASET_CONFIG["ORIGINAL_DIR"], DATASET_CONFIG["GROUND_TRUTH_DIR"], DATASET_CONFIG["TARGET_IMG_SIZE"]
    )
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size], generator=torch.Generator().manual_seed(42))

    num_workers = min(4, os.cpu_count() // 2 if os.cpu_count() and os.cpu_count() > 1 else 1)
    train_loader = DataLoader(train_dataset, batch_size=DATASET_CONFIG["BATCH_SIZE"], shuffle=True, num_workers=num_workers, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=DATASET_CONFIG["BATCH_SIZE"], shuffle=False, num_workers=num_workers, pin_memory=True)

    model = UNet(n_channels_in=3, n_channels_out=3, base_features=64).to(DEVICE)
    l1_loss_fn = nn.L1Loss().to(DEVICE)
    perceptual_loss_fn = VGGPerceptualLoss(feature_layers=DATASET_CONFIG["VGG_FEATURE_LAYERS"]).to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=DATASET_CONFIG["LEARNING_RATE"])
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5, verbose=True)

    print("Starting training...")
    best_val_loss = float('inf')

    for epoch in range(DATASET_CONFIG["NUM_EPOCHS"]):
        model.train()
        epoch_train_loss, epoch_l1_loss_val, epoch_perceptual_loss_val = 0.0, 0.0, 0.0
        
        for batch_idx, (originals, gts) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1} Training")):
            originals, gts = originals.to(DEVICE), gts.to(DEVICE)
            optimizer.zero_grad()
            predictions = model(originals)
            loss_l1 = l1_loss_fn(predictions, gts)
            loss_perceptual = perceptual_loss_fn(predictions, gts)
            total_loss = loss_l1 + DATASET_CONFIG["PERCEPTUAL_LOSS_WEIGHT"] * loss_perceptual
            total_loss.backward()
            optimizer.step()
            epoch_train_loss += total_loss.item()
            epoch_l1_loss_val += loss_l1.item()
            epoch_perceptual_loss_val += loss_perceptual.item()

            if batch_idx > 0 and batch_idx % 100 == 0:
                 tqdm.write(f"  B{batch_idx}/{len(train_loader)} - AvgL1:{epoch_l1_loss_val/(batch_idx+1):.4f}, AvgP:{epoch_perceptual_loss_val/(batch_idx+1):.4f}, AvgTot:{epoch_train_loss/(batch_idx+1):.4f}")

        avg_train_loss = epoch_train_loss / len(train_loader)
        avg_l1_train = epoch_l1_loss_val / len(train_loader)
        avg_perceptual_train = epoch_perceptual_loss_val / len(train_loader)
        print(f"E{epoch+1} Train Total Loss: {avg_train_loss:.4f} (L1: {avg_l1_train:.4f}, P: {avg_perceptual_train:.4f})")

        model.eval()
        epoch_val_loss, epoch_val_l1_loss, epoch_val_perceptual_loss = 0.0, 0.0, 0.0
        with torch.no_grad():
            for originals_val, gts_val in tqdm(val_loader, desc=f"Epoch {epoch+1} Validation"):
                originals_val, gts_val = originals_val.to(DEVICE), gts_val.to(DEVICE)
                predictions_val = model(originals_val)
                val_loss_l1 = l1_loss_fn(predictions_val, gts_val)
                val_loss_perceptual = perceptual_loss_fn(predictions_val, gts_val)
                total_val_loss_batch = val_loss_l1 + DATASET_CONFIG["PERCEPTUAL_LOSS_WEIGHT"] * val_loss_perceptual
                epoch_val_loss += total_val_loss_batch.item()
                epoch_val_l1_loss += val_loss_l1.item()
                epoch_val_perceptual_loss += val_loss_perceptual.item()

        avg_val_loss = epoch_val_loss / len(val_loader)
        avg_val_l1 = epoch_val_l1_loss / len(val_loader)
        avg_val_perceptual = epoch_val_perceptual_loss / len(val_loader)
        print(f"E{epoch+1} Val Total Loss: {avg_val_loss:.4f} (L1: {avg_val_l1:.4f}, P: {avg_val_perceptual:.4f})")
        scheduler.step(avg_val_loss)

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), DATASET_CONFIG["MODEL_SAVE_PATH"])
            print(f"  Saved best model to {DATASET_CONFIG['MODEL_SAVE_PATH']} (Val Total Loss: {best_val_loss:.4f})")
        
        if val_loader and (epoch + 1) % 1 == 0 :
            try:
                originals_sample, gts_sample = next(iter(val_loader))
                originals_sample, gts_sample = originals_sample.to(DEVICE)[:1], gts_sample.to(DEVICE)[:1]
                with torch.no_grad(): prediction_sample = model(originals_sample)
                orig_np = (originals_sample[0].cpu().permute(1,2,0).numpy()*255).clip(0,255).astype(np.uint8)
                gt_np = (gts_sample[0].cpu().permute(1,2,0).numpy()*255).clip(0,255).astype(np.uint8)
                pred_np = (prediction_sample[0].cpu().permute(1,2,0).numpy()*255).clip(0,255).astype(np.uint8)
                comparison_img = np.concatenate((cv2.cvtColor(orig_np,cv2.COLOR_RGB2BGR), cv2.cvtColor(pred_np,cv2.COLOR_RGB2BGR), cv2.cvtColor(gt_np,cv2.COLOR_RGB2BGR)), axis=1)
                sample_save_dir = "training_samples_perceptual"
                os.makedirs(sample_save_dir, exist_ok=True)
                cv2.imwrite(os.path.join(sample_save_dir, f"epoch_{epoch+1}_sample.png"), comparison_img)
                print(f"  Saved val sample to {sample_save_dir}/epoch_{epoch+1}_sample.png")
            except StopIteration: print("  Val loader empty, cannot save sample.")

    print(f"Training finished. Best model saved to {DATASET_CONFIG['MODEL_SAVE_PATH']} (Val Total Loss: {best_val_loss:.4f})")

if __name__ == "__main__":
    if not os.path.exists(DATASET_CONFIG["ORIGINAL_DIR"]) or \
       not os.path.exists(DATASET_CONFIG["GROUND_TRUTH_DIR"]) or \
       not os.listdir(DATASET_CONFIG["ORIGINAL_DIR"]):
        print("Error: Dataset dirs not set up or empty. Run ground truth generation script first.")
    else:
        train_model()

Using device: cuda


Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:01<00:00, 306MB/s] 


Starting training...


Epoch 1 Training:  33%|███▎      | 101/310 [00:46<01:36,  2.17it/s]

  B100/310 - AvgL1:0.3383, AvgP:1.2431, AvgTot:0.4005


...

  B100/310 - AvgL1:0.0022, AvgP:0.1731, AvgTot:0.0109


Epoch 58 Training:  65%|██████▍   | 201/310 [01:33<00:50,  2.15it/s]

  B200/310 - AvgL1:0.0023, AvgP:0.1712, AvgTot:0.0108


Epoch 58 Training:  92%|█████████▏| 286/310 [02:13<00:11,  2.14it/s]


KeyboardInterrupt: 